In [3]:
# Python code to demonstrate "mathematical function application using a parallel cellular algorithm".
# This code provides:
# 1) A vectorized (numpy) implementation (fast, recommended).
# 2) A ThreadPool-based "parallel" cellular update where every cell updates in parallel
#    based on the previous state (synchronous CA update). This mimics a parallel cellular algorithm.
# 3) 1D and 2D examples with sample functions (square, sin, average-with-neighbors).
# The ThreadPool version uses thread-workers so arrays are not repeatedly copied between processes.
# You can modify `cell_rule` to implement other mathematical mappings.
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import math
from typing import Callable, Tuple

print("USN: 1BM23CS245 NAME: PRATHEEKSHA PAI")

def apply_rule_vectorized_1d(state: np.ndarray, rule: Callable[[float, np.ndarray, int], float], radius:int=1) -> np.ndarray:
    """Apply rule to 1D state using vectorized operations when rule is simple.
       This helper expects that `rule` can be expressed as a simple elementwise call;
       if rule depends on neighbors, fall back to `apply_rule_threaded_1d`.
    """
    # If rule is a simple callable that accepts a scalar, we can call it on the whole array
    try:
        return np.vectorize(lambda x: rule(x, None, 0))(state)
    except Exception:
        # Fallback: use threaded version
        return apply_rule_threaded_1d(state, rule, radius)

def apply_rule_threaded_1d(state: np.ndarray, rule: Callable[[float, np.ndarray, int], float], radius:int=1, workers:int=8) -> np.ndarray:
    """Apply rule to 1D state using ThreadPool to mimic parallel cellular update.
       rule(cell_value, neighborhood_array, i) -> new_value
       neighborhood_array is a view of previous state centered at i (with wrap-around).
    """
    n = len(state)
    prev = state.copy()
    def worker(i):
        # gather neighborhood (circular/wrap-around)
        if radius == 0:
            neigh = None
        else:
            idxs = [(i + d) % n for d in range(-radius, radius+1)]
            neigh = prev[idxs]
        return rule(prev[i], neigh, i)
    with ThreadPoolExecutor(max_workers=workers) as ex:
        results = list(ex.map(worker, range(n)))
    return np.array(results, dtype=float)

def apply_rule_threaded_2d(state: np.ndarray, rule: Callable[[float, np.ndarray, Tuple[int,int]], float], radius:int=1, workers:int=8) -> np.ndarray:
    """Apply rule to 2D state using ThreadPool. Synchronous update (all cells read prev state, write new state).
       rule(cell_value, neighborhood_block, (i,j)) -> new_value
       neighborhood_block is a (2*radius+1)x(2*radius+1) view with wrap-around.
    """
    rows, cols = state.shape
    prev = state.copy()
    indices = [(i,j) for i in range(rows) for j in range(cols)]
    def worker(idx):
        i,j = idx
        if radius == 0:
            neigh = None
        else:
            # build neighborhood with wrap-around
            ii = [ (i + di) % rows for di in range(-radius, radius+1) ]
            jj = [ (j + dj) % cols for dj in range(-radius, radius+1) ]
            block = prev[np.ix_(ii, jj)]
            neigh = block
        return rule(prev[i,j], neigh, (i,j))
    with ThreadPoolExecutor(max_workers=workers) as ex:
        results = list(ex.map(worker, indices))
    return np.array(results, dtype=float).reshape(state.shape)

# Example cell rules:
def square_rule(val, neigh, idx):
    return val * val

def sin_rule(val, neigh, idx):
    return math.sin(val)

def avg_neighbors_rule(val, neigh, idx):
    # if neigh is None, return val
    if neigh is None:
        return val
    return float(np.mean(neigh))

def diffusion_rule(val, neigh, idx):
    # simple diffusion: new = val + alpha*(mean(neigh)-val)
    if neigh is None:
        return val
    alpha = 0.5
    return float(val + alpha*(np.mean(neigh) - val))

# Runner functions: iterate steps
def run_1d_example(initial_state: np.ndarray, rule: Callable, steps:int=5, radius:int=1, threaded:bool=True):
    s = initial_state.copy().astype(float)
    print("Initial:", s)
    for t in range(steps):
        if threaded:
            s = apply_rule_threaded_1d(s, rule, radius=radius)
        else:
            s = apply_rule_vectorized_1d(s, rule, radius=radius)
        print(f"Step {t+1}:", np.round(s, 4))
    return s

def run_2d_example(initial_state: np.ndarray, rule: Callable, steps:int=4, radius:int=1, threaded:bool=True):
    s = initial_state.copy().astype(float)
    print("Initial:\n", np.round(s,4))
    for t in range(steps):
        s = apply_rule_threaded_2d(s, rule, radius=radius) if threaded else s  # vectorized 2D neighbor rules are nontrivial here
        print(f"Step {t+1}:\n", np.round(s,4))
    return s

# --- Demonstrations ---
if __name__ == "__main__":
    # 1D demo: square each cell (no neighbors)
    arr1 = np.array([0.5, 1.0, -0.5, 2.0, -1.5])
    print("1D: square_rule (vectorized version recommended)")
    run_1d_example(arr1, square_rule, steps=3, radius=0, threaded=False)
    print("\n1D: diffusion_rule (uses neighbors, threaded parallel update)")
    arr2 = np.array([1.0, 2.0, 0.0, 4.0, 3.0, 0.5])
    run_1d_example(arr2, diffusion_rule, steps=4, radius=1, threaded=True)

    # 2D demo: diffusion on a small grid (mimics heat diffusion)
    grid = np.array([
        [10.0, 10.0, 10.0, 10.0],
        [10.0, 0.0, 0.0, 10.0],
        [10.0, 0.0, 0.0, 10.0],
        [10.0, 10.0, 10.0, 10.0],
    ])
    print("\n2D diffusion demo:")
    run_2d_example(grid, diffusion_rule, steps=5, radius=1, threaded=True)



USN: 1BM23CS245 NAME: PRATHEEKSHA PAI
1D: square_rule (vectorized version recommended)
Initial: [ 0.5  1.  -0.5  2.  -1.5]
Step 1: [0.25 1.   0.25 4.   2.25]
Step 2: [ 0.0625  1.      0.0625 16.      5.0625]
Step 3: [3.90000e-03 1.00000e+00 3.90000e-03 2.56000e+02 2.56289e+01]

1D: diffusion_rule (uses neighbors, threaded parallel update)
Initial: [1.  2.  0.  4.  3.  0.5]
Step 1: [1.0833 1.5    1.     3.1667 2.75   1.    ]
Step 2: [1.1389 1.3472 1.4444 2.7361 2.5278 1.3056]
Step 3: [1.2014 1.3287 1.6435 2.4861 2.3588 1.4815]
Step 4: [1.2693 1.36   1.7315 2.3245 2.2338 1.581 ]

2D diffusion demo:
Initial:
 [[10. 10. 10. 10.]
 [10.  0.  0. 10.]
 [10.  0.  0. 10.]
 [10. 10. 10. 10.]]
Step 1:
 [[9.4444 8.8889 8.8889 9.4444]
 [8.8889 2.7778 2.7778 8.8889]
 [8.8889 2.7778 2.7778 8.8889]
 [9.4444 8.8889 8.8889 9.4444]]
Step 2:
 [[8.9506 8.2716 8.2716 8.9506]
 [8.2716 4.5062 4.5062 8.2716]
 [8.2716 4.5062 4.5062 8.2716]
 [8.9506 8.2716 8.2716 8.9506]]
Step 3:
 [[8.5528 7.9287 7.9287 8.5528]
 